# Подготовка признаков, обучение и тестирование модели

Здесь предсталены основные этапы проекта: подготовка признаков, обучение модели и тестирование ее работы. 

Мы будем использовать набор данных, предоставленный компанией "Союзмультфильм", для классификации видеоклипов по соответствующим проектам.

Конечная цель - максимизировать метрику F1 MACRO.

## 1. Импорт библиотек

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from nltk.corpus import stopwords
import re
import nltk
from nltk.stem import WordNetLemmatizer
from scipy.sparse import hstack
import numpy as np

### Пояснение
Используем популярные библиотеки, такие как `pandas` для работы с данными, `scikit-learn` для машинного обучения и `nltk` для обработки естественного языка. 

## 2. Загрузка данных и ресурсов 

### Загрузка стоп-слов и инициализация лемматизатора

In [2]:
# Загрузка ресурсов NLTK
nltk.download('stopwords')
nltk.download('wordnet')

# Лемматизация и стоп-слова
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('russian'))
stop_words.update(stopwords.words('english'))  # Добавляем стоп-слова для английского

[nltk_data] Downloading package stopwords to /home/karpov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/karpov/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Загрузка данных


In [3]:
# Загрузка данных
train = pd.read_csv('train.csv', lineterminator='\n')
test = pd.read_csv('test.csv', lineterminator='\n')

**Пояснение:** 
- `lineterminator='\n'` используется для корректной обработки новых строк.

## 3. Предварительная обработка данных 

### Функция предварительной обработки текста

In [4]:
# Предварительная обработка текста
def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', text.lower())  # Убираем лишние символы и приводим к нижнему регистру
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]  # Лемматизация и удаление стоп-слов
    return ' '.join(tokens)

### Применение предобработки

In [5]:
# Применение предобработки к тексту
train['text'] = train['text'].apply(lambda x: preprocess_text(str(x)))
test['text'] = test['text'].apply(lambda x: preprocess_text(str(x)))

# Предобработка 'reel_name'
train['reel_name'] = train['reel_name'].apply(lambda x: preprocess_text(str(x)))
test['reel_name'] = test['reel_name'].apply(lambda x: preprocess_text(str(x)))

**Объяснение:** 
- Предварительная обработка текста включает в себя очистку, токенизацию, лемматизацию и удаление стоп-слов из текста. 
- И обучающие, и тестовые данные подвергаются одинаковой предварительной обработке, чтобы обеспечить согласованность.

## 4. Feature Engineering

### Извлечение именованных сущностей

In [6]:
# Функция для извлечения именованных сущностей с использованием регулярных выражений
def extract_named_entities(text):
    # Ищем слова, которые начинаются с заглавной буквы
    names = re.findall(r'\b[A-ZА-ЯЁ][a-zа-яё]*\b', text)
    return ' '.join(names) if names else 'unknown'

# Извлечение именованных сущностей
train['names_str'] = train['text'].apply(lambda x: extract_named_entities(str(x)))
test['names_str'] = test['text'].apply(lambda x: extract_named_entities(str(x)))

**Пояснение:** 
- Здесь мы извлекаем именованные сущности из текста с помощью регулярных выражений, предполагая, что слова, начинающиеся с заглавных букв, являются потенциальными именованными сущностями.

## 5. Векторизация с помощью TF-IDF

In [7]:
# Векторизация текста (TF-IDF) с использованием униграмм
tfidf_text = TfidfVectorizer(max_features=10000, ngram_range=(1, 1))
X_text = tfidf_text.fit_transform(train['text'])

# Векторизация reel_name
tfidf_reel = TfidfVectorizer(max_features=10000, ngram_range=(1, 1))
X_reel = tfidf_reel.fit_transform(train['reel_name'])

# Векторизация имен
tfidf_names = TfidfVectorizer(max_features=5000, ngram_range=(1, 1))
X_names = tfidf_names.fit_transform(train['names_str'])

# Объединение признаков
X = hstack([X_text, X_reel, X_names])
y = train['cartoon']

**Пояснение:**
- Для преобразования текстовых данных в векторы мы используем **TF-IDF**. 
- Текст, название ролика и извлеченные имена векторизуются отдельно, а затем объединяются в единый набор признаков.

## 6. Обучение моделей

### Настройка и обучение модели

In [8]:
# Стратифицированная кросс-валидация с 3 разбиениями
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Определение модели RandomForestClassifier без балансировки классов
model = RandomForestClassifier(max_features='sqrt',
                               min_samples_split=2,
                               random_state=42)

# Обучение модели на всех данных
model.fit(X, y)

RandomForestClassifier(random_state=42)

**Пояснение:**
- В качестве модели мы используем **RandomForestClassifier** со стратифицированной кросс-валидацией для устранения дисбаланса классов. 
- Модель обучается на всех комбинированных признаках.

## 7. Тестирование модели

### Прогнозирование на тестовых данных

In [9]:
# Прогнозирование на тестовых данных
X_test_text = tfidf_text.transform(test['text'])
X_test_reel = tfidf_reel.transform(test['reel_name'])
X_test_names = tfidf_names.transform(test['names_str'])

X_test = hstack([X_test_text, X_test_reel, X_test_names])

test_preds = model.predict(X_test)

### Создаём файл для kaggle

In [10]:
# Создание файла submission
submission = pd.DataFrame({
    'yt_reel_id': test['yt_reel_id'],
    'cartoon': test_preds
})

submission.to_csv('submission_t1.csv', index=False)

## Резюме
В этой тетрадке рассмотрены основные части проекта: 
1. **Подготовка признаков** - включая предварительную обработку текста и извлечение признаков. 
2. **Обучение модели** - использование RandomForestClassifier. 
3. **Тестирование и сохранение результатов** - составление прогнозов на тестовом наборе и сохранение результатов.

Основной целью было создание надежного пайплайна извлечения признаков и обучение модели, которая могла бы точно классифицировать видеоклипы по различным проектам, максимизируя метрику F1 macro.